In [48]:
#open pickle file to dataframe
import pandas as pd
import pickle
with open('../datafiles/filtered_data.pkl', 'rb') as f:
    df = pickle.load(f)

# Set options to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

C:\Users\case4\AppData\Local\Temp\ipykernel_30364\1494833445.py:5: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df = pickle.load(f)


In [70]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import plotly.graph_objs as go


#This function will use the filteredDF for the df 
def clusterScenario(sireCSV, df):
    epd_columns = ['CED', 'BW', 'WW', 'YW', 'MK', 'TM', 'Growth']
    dams_df = df[df['Designation'] == 'Dam'].copy()
    
    #open csv file and convert to dataframe
    sireDf = pd.read_csv(sireCSV)
    #change column name from "Reg No" to "Registration"
    sireDf = sireDf.rename(columns={"Reg No": "Registration Number"})
    sireDf = sireDf.rename(columns={"Milk": "MK"})
    sireDf = sireDf.rename(columns={"Total Maternal": "TM"})
    sireDf = sireDf.rename(columns={"Growth Idx": "Growth"})
    sires_df = sireDf.dropna(subset=epd_columns)
    dams_df = df.dropna(subset=epd_columns)
    
    for col in epd_columns:
        sires_df[col] = pd.to_numeric(sires_df[col], errors='coerce')
        dams_df[col] = pd.to_numeric(dams_df[col], errors='coerce')

    # Select identifier columns and EPDs
    sires_df = sires_df[['Registration Number', 'Name'] + epd_columns]
    sires_df.rename(columns={'Registration Number': 'Sire', 'Name': 'Sire Name'}, inplace=True)

    dams_df = dams_df[['Registration Number'] + epd_columns]
    dams_df.rename(columns={'Registration Number': 'Dam'}, inplace=True)

    # Define desired traits and desired directions
    desired_traits = {
        'CED': 'high',      # Higher Calving Ease Direct is better
        'BW': 'low',   # Moderate Birth Weight
        'WW': 'high',       # Higher Weaning Weight
        'YW': 'high',       # Higher Yearling Weight
        'MK': 'moderate',   # Moderate Milk
        'TM': 'moderate',   # Moderate Total Maternal
        'Growth': 'high'    # Higher Growth
    }

    desired_directions = {
        'CED': 'increase',  # Higher CED is better
        'BW': 'decrease',   # Lower BW is better
        'WW': 'increase',   # Higher WW is better
        'YW': 'increase',   # Higher YW is better
        'MK': 'increase',   # Higher MK is better
        'TM': 'increase',   # Higher TM is better
        'Growth': 'increase' # Higher Growth is better
    }

    # Function to match sires to dam clusters
    def match_sires_to_dam_clusters(sires_df, dams_df, epd_columns, desired_traits, desired_directions, n_clusters=3):
        """
        Clusters dams based on their EPDs, scores sires for each cluster, and identifies the best sire for each cluster.
        """
        # Prepare dam data
        dams_epd = dams_df[['Dam'] + epd_columns].dropna(subset=epd_columns)
        dam_ids = dams_epd['Dam']
        dams_epd_data = dams_epd[epd_columns]
        
        # Standardize dam EPDs
        scaler = StandardScaler()
        dams_epd_scaled = scaler.fit_transform(dams_epd_data)
        
        # Perform clustering on dams
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        cluster_labels = kmeans.fit_predict(dams_epd_scaled)
        dams_epd['Cluster'] = cluster_labels
        
        # Calculate cluster means and EPD gaps
        cluster_means = dams_epd.groupby('Cluster')[epd_columns].mean()
        overall_means = dams_epd[epd_columns].mean()
        epd_gaps = cluster_means - overall_means
        
        # Prepare sire data
        sires_epd = sires_df[['Sire', 'Sire Name'] + epd_columns].dropna(subset=epd_columns)
        sire_ids = sires_epd['Sire']
        sires_epd_data = sires_epd[epd_columns]
        
        # Function to calculate sire score for a cluster
        def sire_cluster_score(sire_epds, cluster_gaps, overall_means, epd_columns, desired_traits, desired_directions):
            score = 0
            for epd in epd_columns:
                trait_pref = desired_traits.get(epd)
                desired_direction = desired_directions.get(epd)
        
                # Calculate the difference between the sire's EPD and the overall mean
                sire_diff = sire_epds[epd] - overall_means[epd]
                gap = cluster_gaps[epd]
        
                if trait_pref == 'high':
                    if desired_direction == 'increase':
                        score += sire_diff * gap
                    elif desired_direction == 'decrease':
                        score -= sire_diff * gap  # Since lower is better
                elif trait_pref == 'moderate':
                    score -= abs(sire_diff) * abs(gap)
                elif trait_pref == 'low':
                    if desired_direction == 'decrease':
                        score += (-sire_diff) * gap
                    elif desired_direction == 'increase':
                        score -= (-sire_diff) * gap
                else:
                    continue
            return score
        
        # Score sires for each cluster
        sire_scores = {}
        for index, sire in sires_epd.iterrows():
            sire_id = sire['Sire']
            sire_epds = sire[epd_columns]
            scores = []
            for cluster_num in cluster_means.index:
                cluster_gap = epd_gaps.loc[cluster_num]
                score = sire_cluster_score(
                    sire_epds=sire_epds,
                    cluster_gaps=cluster_gap,
                    overall_means=overall_means,
                    epd_columns=epd_columns,
                    desired_traits=desired_traits,
                    desired_directions=desired_directions
                )
                scores.append(score)
            sire_scores[sire_id] = scores
        
        # Create a DataFrame of sire scores
        sire_scores_df = pd.DataFrame(sire_scores, index=cluster_means.index)
        
        # Identify best sire for each cluster
        best_sires = {}
        for cluster_num in cluster_means.index:
            best_sire_id = sire_scores_df.loc[cluster_num].idxmax()
            best_sires[cluster_num] = best_sire_id
        
        # Predict offspring EPDs for each cluster
        offspring_epds = {}
        for cluster_num in cluster_means.index:
            best_sire_id = best_sires[cluster_num]
            sire_epds = sires_epd[sires_epd['Sire'] == best_sire_id][epd_columns].iloc[0]
            dam_cluster_mean_epds = cluster_means.loc[cluster_num]
            offspring_epd = (sire_epds + dam_cluster_mean_epds) / 2
            offspring_epds[cluster_num] = offspring_epd
        
        # Prepare dam_clusters DataFrame
        dam_clusters = dams_epd[['Dam', 'Cluster']]
        
        return dam_clusters, best_sires, offspring_epds, cluster_means, overall_means

    # Function to analyze dam clusters
    def analyze_dam_clusters(dams_df, dam_clusters, epd_columns):
        """
        Analyzes each dam cluster to identify EPD deficiencies.
        """
        # Merge dams with their clusters
        dams_clustered = dams_df.merge(dam_clusters, on='Dam')
        
        # Calculate overall means
        overall_means = dams_df[epd_columns].mean()
        
        cluster_analysis = {}
        
        for cluster_num in dam_clusters['Cluster'].unique():
            cluster_dams = dams_clustered[dams_clustered['Cluster'] == cluster_num]
            cluster_mean_epds = cluster_dams[epd_columns].mean()
            epd_gaps = overall_means - cluster_mean_epds  # Positive values indicate deficiencies
            
            # Identify significant deficiencies (e.g., more than 5% below the overall mean)
            deficiencies = epd_gaps[epd_gaps > (0.05 * overall_means)]
            
            cluster_analysis[cluster_num] = {
                'mean_epds': cluster_mean_epds,
                'deficiencies': deficiencies
            }
        
        return cluster_analysis

    # Function to explain sire selection
    def explain_sire_selection(cluster_analysis, best_sires, sires_df, epd_columns):
        """
        Provides explanations for why each sire was selected for a cluster.
        """
        sire_explanations = {}
        
        for cluster_num, sire_id in best_sires.items():
            sire_row = sires_df[sires_df['Sire'] == sire_id].iloc[0]
            sire_epds = sire_row[epd_columns]
            sire_name = sire_row.get('Sire Name', sire_id)
            deficiencies = cluster_analysis[cluster_num]['deficiencies']
            
            # Check how sire's EPDs address deficiencies
            improvements = {}
            for epd in deficiencies.index:
                sire_value = sire_epds[epd]
                dam_cluster_mean = cluster_analysis[cluster_num]['mean_epds'][epd]
                if desired_directions[epd] == 'increase':
                    if sire_value > dam_cluster_mean:
                        improvements[epd] = sire_value - dam_cluster_mean
                elif desired_directions[epd] == 'decrease':
                    if sire_value < dam_cluster_mean:
                        improvements[epd] = dam_cluster_mean - sire_value
            
            sire_explanations[cluster_num] = {
                'sire_id': sire_id,
                'sire_name': sire_name,
                'sire_epds': sire_epds,
                'improvements': improvements
            }
        
        return sire_explanations

    # Function to plot EPD improvements using Plotly
    def plot_epd_improvement(cluster_num, dam_mean_epds, offspring_epds, epd_columns, sire_name):
        """
        Plots the average dam EPDs and predicted offspring EPDs for a cluster using Plotly.
        Includes data labels and the Sire's name in the title.
        """
        # Ensure that the Series are aligned and ordered
        dam_epds = dam_mean_epds[epd_columns].round(2)
        offspring_epds = offspring_epds[epd_columns].round(2)

        # Create traces
        trace1 = go.Bar(
            x=epd_columns,
            y=dam_epds.values,
            name='Dam Average EPDs',
            text=dam_epds.values,
            textposition='auto'
        )
        trace2 = go.Bar(
            x=epd_columns,
            y=offspring_epds.values,
            name='Predicted Offspring EPDs',
            text=offspring_epds.values,
            textposition='auto'
        )

        data = [trace1, trace2]

        # Create layout
        layout = go.Layout(
            title=f'Cluster {cluster_num} EPD Improvement with Sire {sire_name}',
            xaxis=dict(title='EPD Traits'),
            yaxis=dict(title='EPD Values'),
            barmode='group'
        )

        fig = go.Figure(data=data, layout=layout)
        fig.show()

    def display_dams_in_clusters(dam_clusters, dams_df):
        """
        Displays the dams in each cluster.

        Parameters:
        - dam_clusters: DataFrame containing dams and their assigned clusters.
        - dams_df: Original dams DataFrame with additional information if needed.
        """
        # Merge dam_clusters with additional dam information if needed
        dams_clustered = dam_clusters.merge(dams_df, on='Dam', how='left')

        # Group dams by cluster
        clusters = dams_clustered.groupby('Cluster')['Dam'].apply(list)

        # Display dams in each cluster
        for cluster_num, dam_list in clusters.items():
            print(f"\nDams in Cluster {cluster_num}:")
            for dam in dam_list:
                print(f"- {dam}")


    # Number of clusters
    n_clusters = 3  # Adjust based on your data

    # Call the function to match sires to dam clusters
    dam_clusters, best_sires, offspring_epds, cluster_means, overall_means = match_sires_to_dam_clusters(
        sires_df, dams_df, epd_columns, desired_traits, desired_directions, n_clusters=n_clusters
    )

    # Analyze dam clusters
    cluster_analysis = analyze_dam_clusters(dams_df, dam_clusters, epd_columns)

    # Explain sire selection
    sire_explanations = explain_sire_selection(cluster_analysis, best_sires, sires_df, epd_columns)

    # Print detailed explanations and plot results
    for cluster_num in sorted(cluster_analysis.keys()):
        print(f"\n### Cluster {cluster_num} Analysis ###")
        print("Dam Cluster Average EPDs:")
        print(cluster_analysis[cluster_num]['mean_epds'])
        print("\nIdentified EPD Deficiencies (compared to overall means):")
        if not cluster_analysis[cluster_num]['deficiencies'].empty:
            print(cluster_analysis[cluster_num]['deficiencies'])
        else:
            print("No significant deficiencies.")
        
        # Sire selection explanation
        sire_info = sire_explanations[cluster_num]
        print(f"\nSelected Sire for Cluster {cluster_num}: {sire_info['sire_name']} (ID: {sire_info['sire_id']})")
        print("Sire EPDs:")
        print(sire_info['sire_epds'])
        print("\nSire Addresses the Following Deficiencies:")
        if sire_info['improvements']:
            for epd, improvement in sire_info['improvements'].items():
                print(f"- {epd}: Sire improves by {improvement:.2f} units over dam cluster average.")
        else:
            print("Sire does not address any deficiencies directly but was selected based on overall compatibility.")
        
        # Calculate and display improvements
        dam_mean_epds = cluster_analysis[cluster_num]['mean_epds']
        offspring_epd = offspring_epds[cluster_num]
        print("\nExpected Improvement in Offspring EPDs:")
        for epd in epd_columns:
            dam_value = dam_mean_epds[epd]
            offspring_value = offspring_epd[epd]
            improvement = offspring_value - dam_value
            percentage = (improvement / abs(dam_value)) * 100 if dam_value != 0 else 0

            desired_direction = desired_directions[epd]

            # Determine if the change is in the desired direction
            if desired_direction == 'increase':
                is_improvement = improvement > 0
                change_desc = 'increase'
            elif desired_direction == 'decrease':
                is_improvement = improvement < 0
                change_desc = 'decrease'

            # Format the improvement message accordingly
            if is_improvement:
                print(f"- {epd}: Improved by {abs(improvement):.2f} units ({abs(percentage):.2f}% {change_desc})")
            else:
                print(f"- {epd}: No improvement (change of {improvement:.2f} units, {percentage:.2f}% change)")
        
        # Retrieve the Sire's name for plotting
        sire_name = sire_info['sire_name']

        # Plot EPD improvements
        plot_epd_improvement(cluster_num, dam_mean_epds, offspring_epd, epd_columns, sire_name)
    # Display dams in clusters
    display_dams_in_clusters(dam_clusters, dams_df)
    
clusterScenario("../datafiles/sireEpdsOnly.csv", df)


### Cluster 0 Analysis ###
Dam Cluster Average EPDs:
CED        0.280000
BW         2.551429
WW        36.914286
YW        71.685714
MK        26.171429
TM        44.747843
Growth    72.567591
dtype: float64

Identified EPD Deficiencies (compared to overall means):
CED       1.224444
WW        3.278307
YW        3.884656
Growth    4.511918
dtype: float64

Selected Sire for Cluster 0: MGB1854 (ID: AF154061)
Sire EPDs:
CED        -3.506
BW          3.285
WW         54.939
YW         95.207
MK         26.928
TM        54.3975
Growth     99.411
Name: 0, dtype: object

Sire Addresses the Following Deficiencies:
- WW: Sire improves by 18.02 units over dam cluster average.
- YW: Sire improves by 23.52 units over dam cluster average.
- Growth: Sire improves by 26.84 units over dam cluster average.

Expected Improvement in Offspring EPDs:
- CED: No improvement (change of -1.89 units, -676.07% change)
- BW: No improvement (change of 0.37 units, 14.38% change)
- WW: Improved by 9.01 units (24.41

C:\Users\case4\AppData\Local\Temp\ipykernel_30364\3091427072.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




### Cluster 1 Analysis ###
Dam Cluster Average EPDs:
CED       -0.338776
BW         3.026531
WW        48.673469
YW        86.489796
MK        25.530612
TM        49.967010
Growth    89.138076
dtype: float64

Identified EPD Deficiencies (compared to overall means):
CED    1.84322
dtype: float64

Selected Sire for Cluster 1: HEARTBRAND 994Z (ID: AF30264)
Sire EPDs:
CED        10.722
BW          0.925
WW         55.101
YW         96.435
MK         25.079
TM        52.6295
Growth    132.254
Name: 6, dtype: object

Sire Addresses the Following Deficiencies:
- CED: Sire improves by 11.06 units over dam cluster average.

Expected Improvement in Offspring EPDs:
- CED: Improved by 5.53 units (1632.46% increase)
- BW: Improved by 1.05 units (34.72% decrease)
- WW: Improved by 3.21 units (6.60% increase)
- YW: Improved by 4.97 units (5.75% increase)
- MK: No improvement (change of -0.23 units, -0.88% change)
- TM: Improved by 1.33 units (2.66% increase)
- Growth: Improved by 21.56 units (24.18%


### Cluster 2 Analysis ###
Dam Cluster Average EPDs:
CED        4.115686
BW         1.050980
WW        34.294118
YW        67.745098
MK        23.490196
TM        40.670892
Growth    68.590243
dtype: float64

Identified EPD Deficiencies (compared to overall means):
BW        1.106057
WW        5.898475
YW        7.825272
MK        1.435730
TM        4.431134
Growth    8.489266
dtype: float64

Selected Sire for Cluster 2: MGB1854 (ID: AF154061)
Sire EPDs:
CED        -3.506
BW          3.285
WW         54.939
YW         95.207
MK         26.928
TM        54.3975
Growth     99.411
Name: 0, dtype: object

Sire Addresses the Following Deficiencies:
- WW: Sire improves by 20.64 units over dam cluster average.
- YW: Sire improves by 27.46 units over dam cluster average.
- MK: Sire improves by 3.44 units over dam cluster average.
- TM: Sire improves by 13.73 units over dam cluster average.
- Growth: Sire improves by 30.82 units over dam cluster average.

Expected Improvement in Offspring EPDs


Dams in Cluster 0:
- AF112600
- AF168517
- AF173208
- AF178217
- AF178218
- AF178221
- AF178223
- AF178224
- AF197151
- AF204570
- AF217215
- AF251303
- AF251312
- AF295579
- AF295637
- AF295639
- AF295641
- AF295642
- AF295644
- AF295645
- AF295646
- AF295647
- AF295648
- AF295651
- AF295654
- AF295655
- AF295660
- AF323236
- AF323241
- AF323243
- AF323325
- AF330801
- AF36497
- AF62449
- AF91289

Dams in Cluster 1:
- AF106012
- AF168513
- AF170039
- AF178219
- AF178222
- AF178225
- AF197037
- AF197130
- AF197210
- AF209880
- AF21759
- AF251294
- AF251296
- AF251311
- AF274859
- AF274862
- AF291682
- AF295640
- AF295650
- AF295652
- AF295653
- AF295659
- AF295661
- AF295668
- AF323234
- AF323242
- AF323244
- AF323245
- AF323246
- AF323247
- AF323248
- AF323252
- AF323326
- AF323327
- AF323330
- AF323332
- AF323334
- AF330782
- AF330783
- AF330784
- AF330790
- AF330793
- AF330794
- AF330795
- AF330796
- AF330797
- AF330798
- AF330799
- AF91302

Dams in Cluster 2:
- AF106006
- AF11048
